# Defines

In [74]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
I = np.eye(3)
O = np.zeros((3, 3))

# Contmecha formulas

### Rates of change

Velocity gradient:
$$\boldsymbol{l} = \text{grad}\boldsymbol{v}$$
Rate of deformation:
$$\boldsymbol{d} = \text{symm}(\boldsymbol{l}) = \frac{1}{2}(\boldsymbol{l}+\boldsymbol{l}^T)$$
Spin tensor:
$$\boldsymbol{w} = \text{skew}(\boldsymbol{l}) = \frac{1}{2}(\boldsymbol{l}-\boldsymbol{l}^T)$$

# Simple shear

$$
[F] = 
\begin{bmatrix}
    1 & \gamma & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1 \\
\end{bmatrix}
\quad\Rightarrow\quad
[F^{-1}] = 
\begin{bmatrix}
    1 & -\gamma & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1 \\
\end{bmatrix}
$$

In [70]:
N = 1000
t = np.linspace(0,1,N)

gamma  = lambda t: 4*t
dgamma = lambda t: 4

$$
[l] = [\dot{F}F^{-1}] = 
\begin{bmatrix}
    0 & \dot{\gamma}(t) & 0 \\
    0 & 0 & 0 \\
    0 & 0 & 0 \\
\end{bmatrix}
$$

In [71]:
l = lambda t : np.array([[0,dgamma(t),0],[0,0,0],[0,0,0]])

$$
d = \frac{1}{2}(l+l^T) \quad\&\quad w = l - d
$$

In [72]:
d = lambda t: 1/2 * (l(t)+l(t).T)
w = lambda t: l(t) - d(t)

## Radial return algorithm

Hooke's law:
$$
\overset{\circ}{\sigma} = \mathbb{D} : d 
\quad\Rightarrow\quad
\overset{\circ}{\sigma} = \frac{E}{1+\nu}\left(d+\frac{\nu}{1-2\nu}(\text{tr}d)I\right)
$$

In [87]:
E = 200e9
nu = 0.3
G = 76.923e3

D = lambda d: E/(1+nu) * (d + nu/(1-2*nu)*np.trace(d)*I)

Yield curve:
$$
Y(\lambda) = a(\varepsilon_0+\lambda)^n
$$

In [86]:
a = 500e6
eps_0  = 0.00003
n = 0.2

Y = lambda λ : a*(eps_0 + λ)**n

In [88]:
Q = np.tile(I, (N, 1, 1))
sig = np.tile(O, (N, 1, 1))

for n in range(N-1):
    dt = t[n+1] - t[n]
    t_mid = t[n]+dt/2

    # Hughes-Winget
    Q[n+1] = inv(I - 1/2*w(t_mid)*dt) @ (I + 1/2*w(t_mid)*dt) @ Q[n]
    Q_mid = Q[n+1] # Kossa mondta a videóban, hogy nem kell a középső lépés

    deps_tilde = Q_mid.T @ d(t_mid) @ Q_mid

    sig_trial_tilde = sig[n] + D(deps_tilde)
